# Planeación de rutas para la exploración en Marte

In [1]:
!pip install simpleai numpy

  Using cached simpleai-0.8.3.tar.gz (94 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100984 sha256=ea4a0aa33f0727b88f80ee265e1522834420cb34a1b1c4a97548f5b4916bfa80
  Stored in directory: /Users/javier/Library/Caches/pip/wheels/04/84/76/e493a82b60c25f283e4d315ac3b9b9ff44470556da775c58ad
Successfully built simpleai


In [2]:

# Mapeo de (x,y) a (r,c)
import numpy as np
mars_map = np.load('mars_map.npy')
nr, nc = mars_map.shape


In [3]:
cscala = 10.0174

# def mapeo(x, y):
#     r = nr - round(y/cscala)
#     c = round(x/cscala)
#     return r, c

def mapeo(x, y):
    r = max(0, min(nr - round(y / cscala), nr - 1))
    c = max(0, min(round(x / cscala), nc - 1))
    return r, c

In [59]:
#-------------------------------------------------------------------------------
#    Esqueleto de PSA para el problema de ...
#-------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, astar, limited_depth_first
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer

#-------------------------------------------------------------------------------
#   Definición del problema
#-------------------------------------------------------------------------------

class Problem(SearchProblem):
    """
        Clase que es usada para crear el objeto problema. Los estados son representados con
        una estructura en Python que guarde la información descrita en la formulación del PSA.
    """

    def __init__(self, origen, destino, map, change_in_heigth):
        """ Constructor de la clase. Inicializa el problema de acuerdo un conjunto de
            argumentos.

            origen: ([row inicial, col inicial]) 
            destino: ([row final, col final])
            map : Matriz con el mapa de Marte
            change_in_heigth: Valor umbral para el cambio de altura 
a
        """

        # Construye el estado inicial a partir de los argumentos especificados de acuerdo
        # a lo definido en la formulación del PSA.

        # Define las acciones posibles que puede ejecutar el agente
        self.moves = [[1,0], [-1,0], [0,1], [0,-1]] # Abajo, Arriba, Derecha, Izquierda

        # Define el mapa de marte
        self.map = map 
        # Define el tamaño del mapa en filas
        self.nr = map.shape[0]
        # Define el tamaño del mapa en columnas
        self.nc = map.shape[1]
        # Define el cambio de altura máximo permitido
        self.change_in_heigth = change_in_heigth
        
        # Define el estado inicial (obligatorio).
        # Posicion inicial
        origen = (origen[0], origen[1]) 

        # ll# Llama al constructor de su superclase SearchProblem (start = estado inicial).
        SearchProblem.__init__(self, origen)      # Starting position
        self.position = (2850, 6400)
        
        # Define el estado meta (opcional).
        # Posicion final
        self.goal_state = (destino[0], destino[1])
        
    def actions(self, state):
        """
            Regresa una lista con las acciones legales del agente.

            state: [x, y] Estado actual del agente.
        """

        # Determina las acciones legales según el estado recibido y las precondiciones
        # de cada acción posible, para luego colocarlas en una lista de python

        acciones = []

        # Verifica si la acción es legal
        for move in self.moves:
            # Calcula el siguiente estado
            next_state = (state[0] + move[0], state[1] + move[1])
            # Verifica si la acción esta dentro de los limites y el valor no es -1
            if (next_state[0] >= 0 and next_state[0] < self.nr and next_state[1] >= 0 and next_state[1] < self.nc and self.map[next_state[0], next_state[1]] != -1 and abs(self.map[next_state[0], next_state[1]] - self.map[state[0], state[1]]) < self.change_in_heigth):
                acciones.append(move)


        return acciones

    def result(self, state, action):                                   
        ''''
            Regresa el nuevo estado al ejecutar una acción                   
            state: ([row, col) Lista con la posición actual del rover.
            action: [delta_row, delta_col] Lista de enteros indicando el movimiento del rover en los ejes x e y.
        '''              
        # Implementa el modelo de transición para determinar el nuevo estado a par 
        # de aplicar la acción al estado especifica                
        
        # Calcula el nuevo estado
        new_state = (state[0] + action[0], state[1] + action[1])

        return new_state
        

    def is_goal(self, state):
        """
            Determina si se ha llegado a un estado meta.

            state: Estado a ser evaluado.
        """

        # efectúa la prueba de meta para determinar si se ha llegado a un
        # estado deseado

        if(self.goal_state == state):
            return True


    def cost(self, state, action, state2):
        """
            Regresa el costo de ejecutar una acción.

            state: ...
            action: ...
            state2: ...
        """

        # regresa un número que representa el costo de ejecutar action sobre
        # el estado state para producir el estado state2
        
        # Calcula el costo de la acción, en este acaso es 1 para todos los movimientos
        return 1


    def heuristic(self, state):
        """
        Heurística basada en la distancia de Manhattan para el mapa de Marte.

        state: Estado actual del rover (fila, columna).
        """
        # Diferencia absoluta entre las coordenadas del estado actual y el estado meta
        return abs(state[0] - self.goal_state[0]) + abs(state[1] - self.goal_state[1])


# Despliega la secuencia de estados y acciones obtenidas como resultado
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Estado inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de la accion', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de la accion', action)

            print('  ', state)
    else:
        print('Falla: No se pudo resolver el problema')



In [72]:
#-------------------------------------------------------------------------------
#   Solución del problema con diferentes métodos
#-------------------------------------------------------------------------------

change_in_height = 0.29

test_cases = [[mapeo(2850, 6400), mapeo(3150, 6800)],
              [mapeo(10000, 2000), mapeo(10500, 2000)], #500 metros de distancia
              [mapeo(7000, 2000), mapeo(8000, 2000)],  #1000 metros de distancia
              [mapeo(5000, 3000), mapeo(5000, 8000)], #5000 metros de distancia
              [mapeo(4000, 7000), mapeo(14000, 7000)] # 10000 metros de distancia
              ]

# posibles expectadores para las búsquedas
my_viewer = None
#my_viewer = BaseViewer()       # Solo estadísticas
#my_viewer = ConsoleViewer()    # Texto en la consola
#my_viewer = WebViewer()        # Abrir en un browser en la liga http://localhost:8000

# Crea PSAs y los resuelve usando una estrategia de búsqueda mediante
# un algoritmo seleccionado.


## Prueba de algoritmos de búsqueda

#### Primero en anchura

In [65]:
result = breadth_first(Problem(origen=mapeo(2850,6400), destino= mapeo(3150, 6800), map=mars_map, change_in_heigth=change_in_height), graph_search=True, viewer=my_viewer)
print(f'Algoritmo: Breadth First\nEstado de solucion: {result.state}\nCosto: {result.cost}')

Estado de solucion: (1136, 314)
Costo: 505


#### Primero en profundidad


In [66]:
result = depth_first(Problem(origen=mapeo(2850,6400), destino= mapeo(3150, 6800), map=mars_map, change_in_heigth=change_in_height.29), graph_search=True, viewer=my_viewer)
print(f'Algoritmo: Depth First\nEstado de solucion: {result.state}\nCosto:{result.cost}')

Estado de solucion: (1136, 314)
Costo:3501


#### $A^*$

In [67]:
result = astar(Problem(origen=mapeo(2850,6400), destino= mapeo(3150, 6800), map=mars_map, change_in_heigth=change_in_height), graph_search=True, viewer=my_viewer)
print(f'Algoritmo: A-star\nEstado de solucion: {result.state}\nCosto: {result.cost}')

Estado de solucion: (1136, 314)
Costo: 505


#### Costo Uniforme

In [73]:
result = uniform_cost(Problem(origen=mapeo(2850,6400), destino= mapeo(3150, 6800), map=mars_map, change_in_heigth=change_in_height), graph_search=True, viewer=my_viewer)
print(f'Algoritmo: Uniform Cost\nEstado de solucion: {result.state}\nCosto: {result.cost}')

Algoritmo: Uniform Cost
Estado de solucion: (1136, 314)
Costo: 505


### Análisis

- **¿Qué algoritmos lograron encontrar una ruta válida?**  
  Ningún algoritmo logró encontrar una ruta con un límite de diferencia de altura de 0.25. Sin embargo, el límite inferior para que los algoritmos funcionaran fue de 0.29 de diferencia de altura.

- **¿Es necesario utilizar búsquedas informadas para este caso?**  
  No es necesario. De hecho, la búsqueda informada A* encontró una solución con el mismo costo que la búsqueda de costo uniforme y la búsqueda en anchura.

- **¿Qué función heurística resultó adecuada para este problema?**  
  Utilizamos la heurística de distancia de Manhattan:

  ```python
  abs(state[0] - self.goal_state[0]) + abs(state[1] - self.goal_state[1])


## Rendimiento de los algoritmos de búsqueda para rutas cortas y largas

Utilizaremos el metodo $A^*$ para probar los siguientes pares de coordenadas:

**Coordenadas**

Menor a 500 mts: (1461, 621), (1447, 649), 313.5942230092895

Entre 1000 y 5000: (525, 69), (424, 192), 1594.3094144208019

Mayor a 10,000 (1700, 553), (288, 379), 14251.560326935967

In [103]:
result1 = astar(Problem(origen=mapeo(1461,621), destino= mapeo(1464, 649), map=mars_map, change_in_heigth=0.29), graph_search=True, viewer=my_viewer)
try:
    print(f'Algoritmo: A-star\nEstado de solucion: {result1.state}\nCosto: {result1.cost}')
except:
    print('Solucion no encontrada')

Solucion no encontrada


In [86]:
result2 = astar(Problem(origen=mapeo(525,69), destino= mapeo(424, 192), map=mars_map, change_in_heigth=change_in_height), graph_search=True, viewer=my_viewer)
try:
    print(f'Algoritmo: A-star\nEstado de solucion: {result2.state}\nCosto: {result2.cost}')
except:
    print('Solucion no encontrada')

Solucion no encontrada


In [94]:
result3 = depth_first(Problem(origen=mapeo(1700,553), destino= mapeo(288, 379), map=mars_map, change_in_heigth=change_in_height), graph_search=True, viewer=my_viewer)
try:
    print(f'Algoritmo: A-star\nEstado de solucion: {result3.state}\nCosto: {result3.cost}')
except:
    print('Solucion no encontrada')

Solucion no encontrada


### Analisis
- **¿En qué casos el algoritmo es capaz de resolver el problema en un tiempo aceptable?**

    En todos los casos A-estrella no pudo encontrar una solucion. 
    
- **En los casos que el algoritmo no encuentra un resultado, ¿qué acciones se podrían realizar para ayudar al algoritmo a resolver el problema?**

    Ya que A estrella depende de la heuristica, se podría considerar ajustar la funcion. Asimismo, se podria intentar aumentar el limite de altura que el robot puede trasladar. 